In [ ]:
import sys
import numpy as np

import openmm
from openmm import app
from openmm import unit

In [ ]:
field = 'field.xml'
config = 'config.pdb'
#config = 'last.pdb'
#statefile = 'state.xml'

forcefield = app.ForceField(field)
pdb = app.PDBFile(config)

In [ ]:
temperature = 300.0*unit.kelvin
pressure = 1*unit.bar

In [ ]:
modeller = app.Modeller(pdb.topology, pdb.positions)

In [ ]:
print('#  ', modeller.topology.getNumResidues(), 'molecules',
    modeller.topology.getNumAtoms(), 'atoms',
    modeller.topology.getNumBonds(), 'bonds')

box = modeller.topology.getUnitCellDimensions()
print('#   box', box.x, box.y, box.z, 'nm')

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME,
        nonbondedCutoff=12.0*unit.angstrom, constraints=app.HBonds,
        ewaldErrorTolerance=1.0e-5)

In [ ]:
integrator = openmm.LangevinIntegrator(temperature, 5/unit.picosecond, 1*unit.femtosecond)
#integrator = openmm.NoseHooverIntegrator(temperature, 5/unit.picosecond, 1*unit.femtosecond)

In [ ]:
barostat = openmm.MonteCarloBarostat(pressure, temperature)
system.addForce(barostat)

In [ ]:
#platform = openmm.Platform.getPlatformByName('CUDA')
platform = openmm.Platform.getPlatformByName('OpenCL')
#platform = openmm.Platform.getPlatformByName('CPU')
properties = {'Precision': 'single'}

force settings before creating Simulation

In [ ]:
for i, f in enumerate(system.getForces()):
    f.setForceGroup(i)

In [ ]:
sim = app.Simulation(modeller.topology, system, integrator, platform, properties)

In [ ]:
sim.context.setPositions(modeller.positions)
# sim.context.setVelocitiesToTemperature(temperature)

#print('# coordinates and velocities from restart.chk')
#sim.loadCheckpoint('restart.chk')

#print('# coordinates and velocities', statefile)
#sim.loadState(statefile)

#state = sim.context.getState()
#sim.topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())

In [ ]:
platform = sim.context.getPlatform()
print('# platform', platform.getName())
for prop in platform.getPropertyNames():
    print('#   ', prop, platform.getPropertyValue(sim.context, prop))

In [ ]:
state = sim.context.getState(getEnergy=True)
print(state.getPotentialEnergy())

for i, f in enumerate(system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    print(f.getName(), state.getPotentialEnergy())

In [ ]:
print("# Minimizing energy...")
sim.minimizeEnergy()

state = sim.context.getState(getEnergy=True)
print('# PotentielEnergy', state.getPotentialEnergy())

for i, f in enumerate(system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    print('#  ', f.getName(), state.getPotentialEnergy())

In [ ]:
sim.reporters = []
sim.reporters.append(app.StateDataReporter(sys.stdout, 50, step=True,
    speed=True, temperature=True, separator='\t',
    totalEnergy=True, potentialEnergy=True, density=True))

sim.reporters.append(app.DCDReporter('traj.dcd', 50, enforcePeriodicBox=False))

#sim.reporters.append(app.CheckpointReporter('restart.chk', 10000))

In [ ]:
sim.step(10000)

In [ ]:
for i, f in enumerate(system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    print(f.getName(), state.getPotentialEnergy())

In [ ]:
state = sim.context.getState(getPositions=True, getVelocities=True)
coords = state.getPositions()
sim.topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())
app.PDBFile.writeFile(sim.topology, coords, open('last.pdb', 'w'))

In [ ]:
sim.context.setTime(0)
sim.context.setStepCount(0)
sim.saveState('state-eq.xml')

In [ ]:
import mdtraj
import nglview

In [ ]:
trj = mdtraj.load('traj.pdb')
view = nglview.show_mdtraj(trj)
view.add_ball_and_stick('all')
view.add_unitcell()
view.center(zoom=True, resize=True)
view.camera = 'orthographic'
view